In [2]:
import pandas as pd

# 读取CSV文件
data = pd.read_csv('../data/air/城市_20220101-20221231/china_cities_20220101.csv')

# 设定最小和最大阈值（根据文献中不同监测参数的设定）
min_threshold = 0  # 例如：最小值
max_threshold = 1000  # 例如：最大值

# 标记异常数据
data['is_anomaly'] = (data['your_column'] < min_threshold) | (data['your_column'] > max_threshold)

# 输出异常数据
anomalies = data[data['is_anomaly'] == True]
print(anomalies)

KeyError: 'your_column'